## BLS-Table Scraper 
### Shreyas Meher 
#### Last Modified - 6/26/2022

In [ ]:
# import packages needed to run our script
import re
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np

In [ ]:
# function to get user input regarding what BLS table needs to be scraped
def url_func():
    url = input("Please enter the BLS publication that you want to scrape table from: ")
    return url

# one table - https://www.bls.gov/news.release/cpi.t01.htm
# two tables - https://www.bls.gov/regions/southwest/news-release/2022/occupationalemploymentandwages_houston_20220624.htm
# table links - https://www.bls.gov/bls/newsrels.htm#OPLC 
url = url_func()
web_data = urllib.request.urlopen(url).read()

In [ ]:
# create soup object to look for the relevant table tags
soup = BeautifulSoup(web_data, "html.parser")

In [ ]:
# how to deal with multiple tables in the website
tables = soup.find_all("table", class_="regular")
for table in tables:
    # merging the table tags together into one and unwrapping at the same time
    m_tbodies = table.find_all("tbody", recursive=False)
    if len(m_tbodies) > 1:
        main_tbody = m_tbodies[0]
        for tbody in m_tbodies[1:]:
            main_tbody.append(tbody.extract())
            tbody.unwrap()

    # cleaning up footnote markers to make the data more readable
    #ft_markers = []
    #for tfoot in table.find_all("tfoot", recursive=False):
        #for s in tfoot.strings:
            #if " " in s:
                #parts = s.split(maxsplit=1)
                #ft_markers.append(parts[0])
        #tfoot.decompose()

    # remove footnote links
    #for footnoteid in table.select("a > .footnoteid"):
        #footnoteid.parent.decompose()

    # strip footnote markers and dollar signs from table cells
    #if ft_markers:
        #footnote_re = re.compile(r"(\s*(" + "|".join(map(re.escape, ft_markers)) + "))+$")
        #for td in table.find_all("td"):
            #if td.string:
                #td.string = footnote_re.sub("", td.string).lstrip("$")

match = ".+"
if len(tables) > 1:
    match = re.escape(
        input("There are multiple tables on this webpage, which one do you want the script to output? (Eg. Table A)")
    )

In [ ]:
# dataset creation
data_pandas = pd.read_html(str(soup), match=match, flavor="bs4", thousands=",", decimal=".")
df = data_pandas[0]

In [ ]:
# export to csv
df.to_csv(input("Specify .csv filename: "))